In [18]:
#import dependencies/necessary tools
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Reflect Tables into SQLAlchemy ORM

In [10]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [11]:
#reflect hawaii database
Base = automap_base()
Base.prepare(engine, reflect=True)

#view classes
Base.classes.keys()

['measurement', 'station']

In [12]:
#save reference to measurement class
Measurement = Base.classes.measurement

#save reference to station class
Station = Base.classes.station

In [23]:
#create inspector to view columns in classes
inspector = inspect(engine)

In [27]:
#pull column names from each class
m_columns = inspector.get_columns("Measurement")
s_columns = inspector.get_columns("Station")

In [28]:
#viewing measurement columns
for col in m_columns:
    print(col['name'], col['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [29]:
#viewing station columns
for col in s_columns:
    print(col['name'], col['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [13]:
#create session
session = Session(engine)

# Exploratory Precipitation Analysis

In [31]:
#get most recent date
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [40]:
#pulling the date one year before the last date
one_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)

#querying the data from the last 12 months
last_12_months = session.query(Measurement.id, Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).\
    filter(Measurement.date > one_year_ago).all()

#saving the query to a dataframe
last_12_months_df = pd.DataFrame(last_12_months)
last_12_months_df.head()

,id,station,date,prcp,tobs
0,2365,USC00519397,2016-08-24,0.08,79.0
1,2366,USC00519397,2016-08-25,0.08,80.0
2,2367,USC00519397,2016-08-26,0.00,79.0
3,2368,USC00519397,2016-08-27,0.00,77.0
4,2369,USC00519397,2016-08-28,0.01,78.0
